In [1]:
# pip install librosa numpy pandas tensorflow scikit-learn matplotlib seaborn

In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import os

# Load an example heart sound
file_path = "data/heart_sounds/normal180.wav"
y, sr = librosa.load(file_path, sr=22050)  # Load with a fixed sample rate

# Plot the waveform
plt.figure(figsize=(10, 4))
librosa.display.waveshow(y, sr=sr)
plt.title("Heart Sound Waveform")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.show()


In [ ]:
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=22050)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # Extract 13 MFCC features
    mfcc_scaled = np.mean(mfcc.T, axis=0)  # Mean normalization
    return mfcc_scaled

# Example feature extraction
features = extract_features("data/heart_sounds/normal1.wav")
print("Extracted Features:", features)


Extracted Features: [-496.71405     74.617935    63.60462     49.80565     37.305355
   28.786545    24.45515     22.509977    20.628372    17.432358
   13.123776     9.021458     6.4615273]


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load dataset CSV (Assume a dataset with 'filename' and 'label' columns)
data = pd.read_csv("data/heart_sounds/metadata.csv")

# Extract features for all files
X = np.array([extract_features(f"data/heart_sounds/{file}") for file in data["filename"]])
y = data["label"].values

# Encode labels (Healthy = 0, Diseased = 1)
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training Data Shape:", X_train.shape)
print("Testing Data Shape:", X_test.shape)


Training Data Shape: (800, 13)
Testing Data Shape: (200, 13)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Create a simple neural network
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(13,)),  # 13 MFCC features
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')  # Binary Classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=16, validation_data=(X_test, y_test))


c:\Users\gurav\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.2116 - loss: -158.4301 - val_accuracy: 0.1650 - val_loss: -976.6160
Epoch 2/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.1910 - loss: -1822.0294 - val_accuracy: 0.1650 - val_loss: -5559.5293
Epoch 3/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2201 - loss: -8770.3652 - val_accuracy: 0.1650 - val_loss: -20327.5527
Epoch 4/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2034 - loss: -27843.6094 - val_accuracy: 0.1650 - val_loss: -52228.2617
Epoch 5/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2176 - loss: -66249.3594 - val_accuracy: 0.1650 - val_loss: -111045.4766
Epoch 6/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1963 - loss: -143763.2969 - val_accuracy: 0.1650 - val_loss: -203936.1875
Epoch 7/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1784 - loss: -240779.8906 - val_accuracy: 0.1650 - val_loss: -332259.9062
Epoch 8/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step

In [ ]:
# Evaluate on test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")

# Make predictions
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Classification Report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.1562 - loss: -23254592.0000

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1707 - loss: -26167830.0000 
Test Accuracy: 0.17
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        44
           1       0.17      1.00      0.28        33
           2       0.00      0.00      0.00        48
           3       0.00      0.00      0.00        37
           4       0.00      0.00      0.00        38

    accuracy                           0.17       200
   macro avg       0.03      0.20      0.06       200
weighted avg       0.03      0.17      0.05       200



c:\Users\gurav\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\gurav\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\gurav\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
with open("heart_model.tflite", "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\gurav\AppData\Local\Temp\tmpny_zcmxl\assets


INFO:tensorflow:Assets written to: C:\Users\gurav\AppData\Local\Temp\tmpny_zcmxl\assets


Saved artifact at 'C:\Users\gurav\AppData\Local\Temp\tmpny_zcmxl'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 13), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2389395753488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2389395754832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2389395755408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2389395754448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2389395754256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2389395755600: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm  # Progress bar for processing

# Path to dataset
DATASET_PATH = "data/heart_sounds/"
CSV_FILE = os.path.join(DATASET_PATH, "metadata.csv")

# Load metadata file
df = pd.read_csv(CSV_FILE)

# Function to extract MFCC features from an audio file
def extract_features(file_path, max_pad_length=50):
    try:
        y, sr = librosa.load(file_path, sr=22050)  # Load audio file
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)  # Extract 40 MFCCs
        pad_width = max_pad_length - mfcc.shape[1]
        if pad_width > 0:
            mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
        else:
            mfcc = mfcc[:, :max_pad_length]  # Trim to max_pad_length
        return mfcc.flatten()  # Convert to 1D
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return np.zeros(40 * max_pad_length)  # Return empty feature if error

# Process all audio files and extract features
features = []
labels = []

for i, row in tqdm(df.iterrows(), total=len(df)):
    file_name = row["filename"]
    label = row["label"]
    file_path = os.path.join(DATASET_PATH, file_name)

    feature = extract_features(file_path)

    features.append(feature)
    labels.append(label)

# Convert to numpy arrays
X = np.array(features)
y = np.array(labels)

# Save features for future use
np.save(os.path.join(DATASET_PATH, "X.npy"), X)
np.save(os.path.join(DATASET_PATH, "y.npy"), y)

print("✅ Feature extraction complete! Saved X.npy and y.npy.")


100%|██████████| 1000/1000 [00:27<00:00, 36.68it/s]

✅ Feature extraction complete! Saved X.npy and y.npy.


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pickle

# Define dataset path
DATASET_PATH = "data/heart_sounds/"
TRAIN_FOLDER = os.path.join(DATASET_PATH, "train")
TEST_FOLDER = os.path.join(DATASET_PATH, "test")
MODEL_FILE = os.path.join(DATASET_PATH, "trained_model.pkl")

# Create folders if they don’t exist
os.makedirs(TRAIN_FOLDER, exist_ok=True)
os.makedirs(TEST_FOLDER, exist_ok=True)

# Load metadata file (which contains filenames and labels)
metadata_file = os.path.join(DATASET_PATH, "metadata.csv")
df = pd.read_csv(metadata_file)

# Load extracted features
X = np.load(os.path.join(DATASET_PATH, "X.npy"))
y = np.load(os.path.join(DATASET_PATH, "y.npy"))

# Split dataset into training (80%) and testing (20%)
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(
    X, y, df["filename"], test_size=0.2, random_state=42
)

# Save train and test datasets separately
np.save(os.path.join(TRAIN_FOLDER, "X_train.npy"), X_train)
np.save(os.path.join(TRAIN_FOLDER, "y_train.npy"), y_train)
np.save(os.path.join(TEST_FOLDER, "X_test.npy"), X_test)
np.save(os.path.join(TEST_FOLDER, "y_test.npy"), y_test)

# Save filenames for reference
train_indices.to_csv(os.path.join(TRAIN_FOLDER, "train_files.csv"), index=False)
test_indices.to_csv(os.path.join(TEST_FOLDER, "test_files.csv"), index=False)

print("✅ Train and test datasets saved successfully!")
print("✅ Filenames for training and testing saved!")

# Train Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Save trained model
with open(MODEL_FILE, "wb") as model_file:
    pickle.dump(clf, model_file)

print(f"✅ Model trained and saved as {MODEL_FILE}")

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate Model
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Model Accuracy: {accuracy * 100:.2f}%")

# Print Classification Report
print(classification_report(y_test, y_pred))


✅ Train and test datasets saved successfully!
✅ Filenames for training and testing saved!
✅ Model trained and saved as data/heart_sounds/trained_model.pkl
✅ Model Accuracy: 99.00%
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        44
           1       0.97      1.00      0.99        33
           2       1.00      1.00      1.00        48
           3       1.00      0.97      0.99        37
           4       0.97      0.97      0.97        38

    accuracy                           0.99       200
   macro avg       0.99      0.99      0.99       200
weighted avg       0.99      0.99      0.99       200



In [ ]:
import pickle
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np

# Define dataset path
DATASET_PATH = "data/heart_sounds/"
MODEL_FILE = os.path.join(DATASET_PATH, "trained_model.pkl")

# Load extracted features
X = np.load(os.path.join(DATASET_PATH, "X.npy"))
y = np.load(os.path.join(DATASET_PATH, "y.npy"))

# Split dataset into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Save trained model to a file
with open(MODEL_FILE, "wb") as model_file:
    pickle.dump(clf, model_file)

print(f"✅ Model saved successfully as {MODEL_FILE}")


✅ Model saved successfully as data/heart_sounds/trained_model.pkl


In [ ]:
import shutil
import os

def copy_trained_model(source_folder, destination_folder, filename="trained_model.pkl"):
    """
    Copy the trained_model.pkl file from the source folder to the destination folder.

    :param source_folder: Path to the folder containing the trained_model.pkl file.
    :param destination_folder: Path to the folder where the file should be copied.
    :param filename: Name of the model file (default is 'trained_model.pkl').
    """
    source_path = os.path.join(source_folder, filename)
    destination_path = os.path.join(destination_folder, filename)

    if os.path.exists(source_path):
        shutil.copy(source_path, destination_path)
        print(f"Copied {filename} to {destination_folder}")
    else:
        print(f"Error: {filename} not found in {source_folder}")

# Example Usage
source_folder = "./data/heart_sounds"  # Change this to the actual source directory
destination_folder = "./data/"  # Change this to the actual destination directory

copy_trained_model(source_folder, destination_folder)

Copied trained_model.pkl to ./data/


In [ ]:
import librosa
import numpy as np
import os

# Define dataset path
DATASET_PATH = "data/heart_sounds/"
MODEL_FILE = os.path.join(DATASET_PATH, "trained_model.pkl")  # If saving the model
NEW_AUDIO_FILE = "data/test/rpa.wav"  # Provide your test file

# Function to extract MFCC features from a new audio file
def extract_features_from_audio(file_path, max_pad_length=50):
    try:
        y, sr = librosa.load(file_path, sr=22050)  # Load audio file
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)  # Extract 40 MFCCs
        pad_width = max_pad_length - mfcc.shape[1]
        if pad_width > 0:
            mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
        else:
            mfcc = mfcc[:, :max_pad_length]  # Trim to max_pad_length
        return mfcc.flatten()  # Convert to 1D
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return np.zeros(40 * max_pad_length)  # Return empty feature if error

# Extract features from the new audio file
new_audio_features = extract_features_from_audio(NEW_AUDIO_FILE)

# Reshape for prediction
X_new = np.array(new_audio_features).reshape(1, -1)

# Load trained model (if needed)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pickle

# Load saved features
X = np.load(os.path.join(DATASET_PATH, "train/X_train.npy"))
y = np.load(os.path.join(DATASET_PATH, "train/y_train.npy"))

# Split dataset (Same as before)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on the new audio file
predicted_label = clf.predict(X_new)[0]

# Define label meanings
label_map = {
    0: "Normal Heart Sound",
    1: "Disease 1",
    2: "Disease 2",
    3: "Disease 3",
    4: "Disease 4"
}

# Print the prediction
print(f"✅ The model predicts: {label_map[predicted_label]}")
